In [1]:
import pandas as pd

PATH = '/home/kai/data/kaggle/homecredit/'
prev = pd.read_csv(PATH + 'previous_application.csv')
train = pd.read_csv(PATH + 'application_train.csv')
test = pd.read_csv(PATH + 'application_test.csv')
prev.head(2)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0


In [2]:
categorical_cols = [col for col in prev.columns.values if prev[col].dtype == 'object']

In [3]:
categorical_cols

['NAME_CONTRACT_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'FLAG_LAST_APPL_PER_CONTRACT',
 'NAME_CASH_LOAN_PURPOSE',
 'NAME_CONTRACT_STATUS',
 'NAME_PAYMENT_TYPE',
 'CODE_REJECT_REASON',
 'NAME_TYPE_SUITE',
 'NAME_CLIENT_TYPE',
 'NAME_GOODS_CATEGORY',
 'NAME_PORTFOLIO',
 'NAME_PRODUCT_TYPE',
 'CHANNEL_TYPE',
 'NAME_SELLER_INDUSTRY',
 'NAME_YIELD_GROUP',
 'PRODUCT_COMBINATION']

In [4]:
for col in categorical_cols:
    print(col)
    tmp = set(prev[col].values)
    print(tmp)
    for cat in tmp:
        print(cat)
        print('ratio,',(prev[col]==cat).sum()/len(prev))


NAME_CONTRACT_TYPE
{'XNA', 'Cash loans', 'Consumer loans', 'Revolving loans'}
XNA
ratio, 0.0002071590826085759
Cash loans
ratio, 0.4475791724892738
Consumer loans
ratio, 0.4365614226679934
Revolving loans
ratio, 0.11565224576012414
WEEKDAY_APPR_PROCESS_START
{'SATURDAY', 'MONDAY', 'WEDNESDAY', 'TUESDAY', 'SUNDAY', 'THURSDAY', 'FRIDAY'}
SATURDAY
ratio, 0.1440719572461972
MONDAY
ratio, 0.15181108528607712
WEDNESDAY
ratio, 0.15268103368789868
TUESDAY
ratio, 0.1527456960605048
SUNDAY
ratio, 0.09864065323365748
THURSDAY
ratio, 0.1491419662390568
FRIDAY
ratio, 0.15090760824660793
FLAG_LAST_APPL_PER_CONTRACT
{'Y', 'N'}
Y
ratio, 0.9949257999274345
N
ratio, 0.005074200072565552
NAME_CASH_LOAN_PURPOSE
{'Furniture', 'Buying a new car', 'Other', 'Everyday expenses', 'Building a house or an annex', 'Hobby', 'Car repairs', 'Buying a garage', 'Refusal to name the goal', 'XAP', 'Buying a used car', 'Money for a third person', 'Repairs', 'XNA', 'Purchase of electronic equipment', 'Urgent needs', 'Busin

ratio, 0.007545140922061484
Cash
ratio, 0.17122955501510584
POS industry with interest
ratio, 0.05917385436836238
Cash Street: low
ratio, 0.020257284395891785
Cash X-Sell: middle
ratio, 0.08614644590453678


In [2]:
def minus_name(col1, col2): return col1 + '_minus_' + col2
def minus(df, col1, col2): return df[col1] - df[col2]

def ratio_name(col1, col2): return col1 + '_divide_' + col2
def ratio(df, col1, col2): return df[col1] / (df[col2] + 1)

def positive_count(df, gp_col, col):
    group = (df[col] > 0).astype('int8')
    group = pd.concat([df[gp_col], group], axis=1).groupby(gp_col)[[col]].sum().reset_index().rename(index=str, columns={col: 'positivecount_'+'_'.join([gp_col, col])})
    return group.astype({gp_col: df[gp_col].dtype})

def count(df, gp_col, col):
    group = df[[gp_col,col]].groupby(gp_col)[[col]].count().reset_index().rename(index=str, columns={col:'count_'+gp_col})
    return group.astype({gp_col: df[gp_col].dtype})

def numerical(df, gp_col, col, agg_fun):
    _df = df.groupby(gp_col)[[col]].agg(agg_fun)
    
    columns = []
    for x in _df.columns.levels[0]:
        for y in _df.columns.levels[1]:
            columns.append('_'.join([x, y]))
    _df.columns = columns
    return _df.reset_index().astype({gp_col: df[gp_col].dtype})

def one_hot(df, gp_col, col):
    return pd.concat([df[gp_col], pd.get_dummies(df[col], prefix='onehot_' + col)], axis=1).groupby(gp_col).sum().reset_index().astype({gp_col: df[gp_col].dtype})

'done'

'done'

In [3]:
prev.columns.values

array(['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'AMT_ANNUITY',
       'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT',
       'AMT_GOODS_PRICE', 'WEEKDAY_APPR_PROCESS_START',
       'HOUR_APPR_PROCESS_START', 'FLAG_LAST_APPL_PER_CONTRACT',
       'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT',
       'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED',
       'NAME_CASH_LOAN_PURPOSE', 'NAME_CONTRACT_STATUS', 'DAYS_DECISION',
       'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON', 'NAME_TYPE_SUITE',
       'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO',
       'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE', 'SELLERPLACE_AREA',
       'NAME_SELLER_INDUSTRY', 'CNT_PAYMENT', 'NAME_YIELD_GROUP',
       'PRODUCT_COMBINATION', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE',
       'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION',
       'NFLAG_INSURED_ON_APPROVAL'], dtype=object)

In [4]:
ignore = ['SELLERPLACE_AREA', 'PRODUCT_COMBINATION']
not_ignore = [x for x in prev.columns if x not in ignore]
prev = prev[not_ignore]

In [5]:
merged_col = 'SK_ID_CURR'
df = pd.concat([train['SK_ID_CURR'], test['SK_ID_CURR']], ignore_index=True).to_frame()

# minus
name1 = minus_name('DAYS_TERMINATION', 'DAYS_FIRST_DRAWING')
prev[name1] = minus(prev, 'DAYS_TERMINATION', 'DAYS_FIRST_DRAWING')
name2 = minus_name('DAYS_FIRST_DUE', 'DAYS_FIRST_DRAWING')
prev[name2] = minus(prev, 'DAYS_FIRST_DUE', 'DAYS_FIRST_DRAWING')
name3 = minus_name('DAYS_TERMINATION', 'DAYS_LAST_DUE')
prev[name3] = minus(prev, 'DAYS_TERMINATION', 'DAYS_LAST_DUE')
print('minus')

# deal with some cat feat
prev['FLAG_NAME_CASH_LOAN_PURPOSE'] = (prev['NAME_CASH_LOAN_PURPOSE']=='XNA') + (prev['NAME_CASH_LOAN_PURPOSE']=='XAP')*2
prev['FLAG_NAME_GOODS_CATEGORY'] = (prev['NAME_GOODS_CATEGORY'] == 'XNA') + 0
print('deal')

# ratio
numerator = [x for x in prev.columns if 'AMT' in x]
denominator = [x for x in prev.columns if 'DAYS' in x or 'AMT' in x] + [name1, name2, name3]
for x in numerator:
    for y in denominator:
        if (x != y) and (ratio_name(y, x) not in prev.columns):
            prev[ratio_name(x, y)] = ratio(prev, x, y)
print('ratio')
        
# one hot
categorical = ['NAME_CONTRACT_TYPE', 'FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY', 'NAME_CONTRACT_STATUS',
               'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON', 'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE',
               'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE', 'NFLAG_INSURED_ON_APPROVAL', 
               'NAME_SELLER_INDUSTRY', 'NAME_YIELD_GROUP', 'WEEKDAY_APPR_PROCESS_START'] + \
              ['FLAG_NAME_CASH_LOAN_PURPOSE', 'FLAG_NAME_GOODS_CATEGORY']
for x in categorical:
    print(x)
    df = df.merge(one_hot(prev, merged_col, x), on=merged_col, how='left')
    
# count
df = df.merge(count(prev, merged_col, 'NAME_CONTRACT_TYPE'), on=merged_col, how='left')
count_name = [x for x in df.columns if 'count_' in x][0]
print('count')

# count ratio
for x in df.columns:
    if 'positivecount_' in x or 'onehot_' in x:
        print(x)
        df[ratio_name(x, count_name)] = ratio(df, x, count_name)

# numerical
ignored = ['SK_ID_CURR', 'SK_ID_PREV', 'NAME_CASH_LOAN_PURPOSE', 'NAME_GOODS_CATEGORY'] + categorical
for x in set(prev.columns) - set(ignored):
    print(x)
    df = df.merge(numerical(prev, merged_col, x, ['mean', 'median', 'max', 'min', 'sum']), on=merged_col, how='left')
    
columns = []
for x in df.columns:
    tmp = 'prev_' + x if x != merged_col else x
    columns.append(tmp)
df.columns = columns

'done'

minus
deal
ratio
NAME_CONTRACT_TYPE
FLAG_LAST_APPL_PER_CONTRACT
NFLAG_LAST_APPL_IN_DAY
NAME_CONTRACT_STATUS
NAME_PAYMENT_TYPE
CODE_REJECT_REASON
NAME_TYPE_SUITE
NAME_CLIENT_TYPE
NAME_PORTFOLIO
NAME_PRODUCT_TYPE
CHANNEL_TYPE
NFLAG_INSURED_ON_APPROVAL
NAME_SELLER_INDUSTRY
NAME_YIELD_GROUP
WEEKDAY_APPR_PROCESS_START
FLAG_NAME_CASH_LOAN_PURPOSE
FLAG_NAME_GOODS_CATEGORY
count
onehot_NAME_CONTRACT_TYPE_Cash loans
onehot_NAME_CONTRACT_TYPE_Consumer loans
onehot_NAME_CONTRACT_TYPE_Revolving loans
onehot_NAME_CONTRACT_TYPE_XNA
onehot_FLAG_LAST_APPL_PER_CONTRACT_N
onehot_FLAG_LAST_APPL_PER_CONTRACT_Y
onehot_NFLAG_LAST_APPL_IN_DAY_0
onehot_NFLAG_LAST_APPL_IN_DAY_1
onehot_NAME_CONTRACT_STATUS_Approved
onehot_NAME_CONTRACT_STATUS_Canceled
onehot_NAME_CONTRACT_STATUS_Refused
onehot_NAME_CONTRACT_STATUS_Unused offer
onehot_NAME_PAYMENT_TYPE_Cash through the bank
onehot_NAME_PAYMENT_TYPE_Cashless from the account of the employer
onehot_NAME_PAYMENT_TYPE_Non-cash from your account
onehot_NAME_PAYMENT_T

'done'

In [6]:
print(df.columns)
df.shape

Index(['SK_ID_CURR', 'prev_onehot_NAME_CONTRACT_TYPE_Cash loans',
       'prev_onehot_NAME_CONTRACT_TYPE_Consumer loans',
       'prev_onehot_NAME_CONTRACT_TYPE_Revolving loans',
       'prev_onehot_NAME_CONTRACT_TYPE_XNA',
       'prev_onehot_FLAG_LAST_APPL_PER_CONTRACT_N',
       'prev_onehot_FLAG_LAST_APPL_PER_CONTRACT_Y',
       'prev_onehot_NFLAG_LAST_APPL_IN_DAY_0',
       'prev_onehot_NFLAG_LAST_APPL_IN_DAY_1',
       'prev_onehot_NAME_CONTRACT_STATUS_Approved',
       ...
       'prev_AMT_GOODS_PRICE_divide_DAYS_FIRST_DRAWING_mean',
       'prev_AMT_GOODS_PRICE_divide_DAYS_FIRST_DRAWING_median',
       'prev_AMT_GOODS_PRICE_divide_DAYS_FIRST_DRAWING_max',
       'prev_AMT_GOODS_PRICE_divide_DAYS_FIRST_DRAWING_min',
       'prev_AMT_GOODS_PRICE_divide_DAYS_FIRST_DRAWING_sum',
       'prev_RATE_DOWN_PAYMENT_mean', 'prev_RATE_DOWN_PAYMENT_median',
       'prev_RATE_DOWN_PAYMENT_max', 'prev_RATE_DOWN_PAYMENT_min',
       'prev_RATE_DOWN_PAYMENT_sum'],
      dtype='object', length=5

(356255, 536)

In [7]:
df.to_pickle(PATH + 'inter/prev2curr.pkl')
'done'

'done'

In [6]:
prev.isnull().sum()/len(prev)

SK_ID_PREV                     0.000000e+00
SK_ID_CURR                     0.000000e+00
NAME_CONTRACT_TYPE             0.000000e+00
AMT_ANNUITY                    2.228667e-01
AMT_APPLICATION                0.000000e+00
AMT_CREDIT                     5.987257e-07
AMT_DOWN_PAYMENT               5.363648e-01
AMT_GOODS_PRICE                2.308177e-01
WEEKDAY_APPR_PROCESS_START     0.000000e+00
HOUR_APPR_PROCESS_START        0.000000e+00
FLAG_LAST_APPL_PER_CONTRACT    0.000000e+00
NFLAG_LAST_APPL_IN_DAY         0.000000e+00
RATE_DOWN_PAYMENT              5.363648e-01
RATE_INTEREST_PRIMARY          9.964370e-01
RATE_INTEREST_PRIVILEGED       9.964370e-01
NAME_CASH_LOAN_PURPOSE         0.000000e+00
NAME_CONTRACT_STATUS           0.000000e+00
DAYS_DECISION                  0.000000e+00
NAME_PAYMENT_TYPE              0.000000e+00
CODE_REJECT_REASON             0.000000e+00
NAME_TYPE_SUITE                4.911975e-01
NAME_CLIENT_TYPE               0.000000e+00
NAME_GOODS_CATEGORY            0

In [11]:
x = prev[['RATE_INTEREST_PRIMARY','RATE_INTEREST_PRIVILEGED']].dropna()

In [12]:
x.shape

(5951, 2)

In [14]:
(x['RATE_INTEREST_PRIMARY']<x['RATE_INTEREST_PRIVILEGED']).sum()

5824

In [18]:
index_interest = x.index
x

,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED
0,0.182832,0.867336
30,0.196914,0.867336
506,0.189136,0.835095
598,0.695668,0.568710
659,0.191757,0.845137
962,0.189122,0.835095
1027,0.193330,0.852537
1028,0.160716,0.715645
1274,0.196900,0.867336
1360,0.196914,0.867336


In [21]:
index_interest
index_interest = prev.loc[index_interest]

In [23]:
set(index_interest['NAME_CONTRACT_STATUS'])

{'Approved'}

In [24]:
prev.columns

Index(['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'AMT_ANNUITY',
       'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE',
       'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY',
       'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY',
       'RATE_INTEREST_PRIVILEGED', 'NAME_CASH_LOAN_PURPOSE',
       'NAME_CONTRACT_STATUS', 'DAYS_DECISION', 'NAME_PAYMENT_TYPE',
       'CODE_REJECT_REASON', 'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE',
       'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE',
       'CHANNEL_TYPE', 'SELLERPLACE_AREA', 'NAME_SELLER_INDUSTRY',
       'CNT_PAYMENT', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION',
       'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION',
       'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL'],
      dtype='object')

In [25]:
amt_list = ['AMT_ANNUITY',
       'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE']

In [27]:
for col in amt_list:
    print(col)
    print((prev[col]== np.nan).sum(), 'num of nan')
    print((prev[col]== 0).sum()/len(prev), 'pct of zeros')

AMT_ANNUITY
0 num of nan
0.0009801139255209213 pct of zeros
AMT_APPLICATION
0 num of nan
0.23494115125367168 pct of zeros
AMT_CREDIT
0 num of nan
0.20163164720209506 pct of zeros
AMT_DOWN_PAYMENT
0 num of nan
0.22144108479512206 pct of zeros
AMT_GOODS_PRICE
0 num of nan
0.004112646642885283 pct of zeros
